In [1]:
!pip install tkcalendar

import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import Calendar
import sqlite3


In [2]:
#Setup SQLite databse to store reminders
def setup_database():
    conn = sqlite3.connect("reminders.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS reminders (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT NOT NULL,
            reminder TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()

In [3]:
#Add reminder to the database
def add_reminder(date, reminder_text):
    conn = sqlite3.connect("reminders.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO reminders (date, reminder) VALUES (?, ?)", (date, reminder_text))
    conn.commit()
    conn.close()


In [4]:
#Fetch reminders for a specific date
def fetch_reminders(date):
    conn = sqlite3.connect("reminders.db")
    cursor = conn.cursor()
    cursor.execute("SELECT reminder FROM reminders WHERE date = ?", (date,))
    reminders = cursor.fetchall()
    conn.close()
    return [r[0] for r in reminders]

In [5]:
#Main class
class CalendarApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Calendar with Reminders")

        # Calendar widget
        self.calendar = Calendar(root, selectmode='day', date_pattern="yyyy-mm-dd")
        self.calendar.pack(pady=20)

        # Add Reminder Section
        self.reminder_label = tk.Label(root, text="Add Reminder:")
        self.reminder_label.pack()

        self.reminder_entry = tk.Entry(root, width=50)
        self.reminder_entry.pack(pady=5)

        self.add_button = tk.Button(root, text="Add Reminder", command=self.add_reminder)
        self.add_button.pack(pady=5)

        # View Reminders Section
        self.view_button = tk.Button(root, text="View Reminders", command=self.view_reminders)
        self.view_button.pack(pady=5)

        self.reminders_list = tk.Listbox(root, width=50, height=10)
        self.reminders_list.pack(pady=10)

    def add_reminder(self):
        date = self.calendar.get_date()
        reminder_text = self.reminder_entry.get()

        if reminder_text.strip():
            add_reminder(date, reminder_text)
            messagebox.showinfo("Success", "Reminder added successfully!")
            self.reminder_entry.delete(0, tk.END)
        else:
            messagebox.showwarning("Input Error", "Please enter a reminder.")

    def view_reminders(self):
        date = self.calendar.get_date()
        reminders = fetch_reminders(date)
        self.reminders_list.delete(0, tk.END)

        if reminders:
            for reminder in reminders:
                self.reminders_list.insert(tk.END, reminder)
        else:
            self.reminders_list.insert(tk.END, "No reminders for this date.")



In [ ]:
# Main Execution
if __name__ == "__main__":
    setup_database()
    root = tk.Tk()
    app = CalendarApp(root)
    root.mainloop()